In [ ]:
# # --- 3.2. Proses Evaluasi untuk Setiap Siswa (Fokus pada RAG) ---

# print("\nMemulai proses evaluasi RAG untuk jawaban siswa...")
# all_evaluations = {}

# # Filter untuk 5 mahasiswa pertama sesuai permintaan Anda
# students_to_evaluate = ['M1', 'M2', 'M3', 'M4', 'M5']
# # Mengambil data dari variabel student_answers_data yang sudah dimuat dari JSON
# filtered_answers = [ans for ans in student_answers_data if ans["student_id"] in students_to_evaluate]

# # Loop utama untuk evaluasi
# for student_ans in tqdm(filtered_answers, desc="Mengevaluasi Jawaban"):
#     student_id = student_ans["student_id"]
#     question_id = student_ans["question_id"]

#     if student_id not in all_evaluations:
#         all_evaluations[student_id] = {}

#     try:
#         # Mencari teks pertanyaan yang sesuai
#         question_text = next(q["question_text"] for q in question_data if q["question_id"] == question_id)
#     except StopIteration:
#         print(f"Peringatan: Melewati jawaban untuk {student_id} karena question_id '{question_id}' tidak ditemukan di data soal.")
#         continue

#     # --- Proses RAG ---
#     # 1. Retrieval: Dapatkan embedding dari pertanyaan dan cari chunk yang relevan
#     query_embedding = get_embeddings([question_text])[0]
#     retrieved_chunks = retrieve_knowledge(query_embedding)

#     # 2. Generation: Hasilkan evaluasi menggunakan Gemma dengan konteks yang ditemukan
#     gemma_evaluation = generate_gemma_response(
#         question_text,
#         student_ans["original_text"],
#         retrieved_chunks
#     )

#     # --- Gabungkan semua hasil ke dalam dictionary ---
#     all_evaluations[student_id][question_id] = {
#         "original_student_answer": student_ans["original_text"],
#         "retrieved_context": "\n\n".join([c['text'] for c in retrieved_chunks]),
#         "simple_o_score": None,  # Diatur ke None (akan menjadi null di JSON)
#         "gemma_evaluation": gemma_evaluation
#     }

# print("\nProses evaluasi RAG selesai.")

In [ ]:
# --- 3.2. Proses Evaluasi (UJI COBA untuk 3 Jawaban) ---

# (1) Impor library tambahan
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os

print("\nMemulai proses evaluasi RAG (UJI COBA)...")
all_evaluations = {}
timing_data = []

# (2) AMBIL 3 JAWABAN PERTAMA UNTUK UJI COBA
# Modifikasi utama ada di sini. Kita mengambil slice `[:3]` dari data utama.
test_answers = student_answers_data[:3]
print(f"Memulai uji coba dengan {len(test_answers)} jawaban pertama...")

# (3) Loop utama untuk evaluasi HANYA pada data uji coba
for student_ans in tqdm(test_answers, desc="Mengevaluasi 3 Jawaban Uji Coba"):
    # Catat waktu mulai
    start_time = time.time()

    student_id = student_ans["student_id"]
    question_id = student_ans["question_id"]

    if student_id not in all_evaluations:
        all_evaluations[student_id] = {}

    try:
        question_text = next(
            q["question_text"] for q in question_data if q["question_id"] == question_id
        )
    except StopIteration:
        print(
            f"Peringatan: Melewati jawaban untuk {student_id} karena question_id '{question_id}' tidak ditemukan."
        )
        continue

    # --- Proses RAG ---
    query_embedding = get_embeddings([question_text])[0]
    retrieved_chunks = retrieve_knowledge(query_embedding)
    gemma_evaluation = generate_gemma_response(
        question_text, student_ans["original_text"], retrieved_chunks
    )

    # --- Gabungkan hasil ---
    all_evaluations[student_id][question_id] = {
        "original_student_answer": student_ans["original_text"],
        "retrieved_context": "\n\n".join([c["text"] for c in retrieved_chunks]),
        "simple_o_score": None,
        "gemma_evaluation": gemma_evaluation,
    }

    # (4) Catat waktu selesai dan simpan durasi
    end_time = time.time()
    evaluation_time = end_time - start_time
    timing_data.append(
        {
            "student_id": student_id,
            "question_id": question_id,
            "evaluation_time_seconds": evaluation_time,
        }
    )

print("\nProses evaluasi RAG (UJI COBA) selesai.")

# Menyimpan hasil evaluasi dari uji coba
output_eval_path = os.path.join(KB_PATH, "test_3_evaluations.json")
with open(output_eval_path, "w", encoding="utf-8") as f:
    json.dump(all_evaluations, f, ensure_ascii=False, indent=4)
print(f"\nHasil evaluasi uji coba disimpan di: {output_eval_path}")

# ==============================================================================
# BAGIAN ANALISIS WAKTU (dari hasil uji coba)
# ==============================================================================
if timing_data:
    print("\n--- Analisis Waktu Evaluasi (dari hasil uji coba) ---")

    timing_df = pd.DataFrame(timing_data)
    timing_csv_path = os.path.join(KB_PATH, "test_3_timing_report.csv")
    timing_df.to_csv(timing_csv_path, index=False)
    print(f"Laporan waktu evaluasi uji coba telah disimpan di: {timing_csv_path}")

    print("\nCuplikan Laporan Waktu:")
    print(timing_df.head())

    print("\nStatistik Waktu Evaluasi (detik):")
    print(timing_df["evaluation_time_seconds"].describe())

    print("\nMembuat visualisasi plot...")
    plt.style.use("seaborn-v0_8-whitegrid")
    fig, axes = plt.subplots(1, 1, figsize=(10, 6))

    sns.barplot(
        x=timing_df.index,
        y=timing_df["evaluation_time_seconds"],
        ax=axes,
        palette="rocket",
    )
    axes.set_title("Waktu Evaluasi untuk 3 Jawaban Pertama", fontsize=16)
    axes.set_xlabel("Indeks Jawaban Uji Coba", fontsize=12)
    axes.set_ylabel("Waktu (detik)", fontsize=12)

    plt.tight_layout()
    plot_path = os.path.join(KB_PATH, "test_3_timing_plot.png")
    plt.savefig(plot_path)
    print(f"Plot visualisasi waktu uji coba telah disimpan di: {plot_path}")
    plt.show()
else:
    print("\nTidak ada data waktu yang tercatat dari uji coba.")

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

print("Mempersiapkan wrapper LLM yang kompatibel dengan LangChain...")

# 1. Buat text-generation pipeline dari Hugging Face
gemma_pipeline = pipeline(
    "text-generation",
    model=gemma_model,
    tokenizer=gemma_tokenizer,
    max_new_tokens=512,
    torch_dtype=torch.bfloat16,
    batch_size=32,
)

# 2. Bungkus pipeline tersebut menggunakan HuggingFacePipeline dari LangChain
# Objek 'langchain_llm' inilah yang akan kita berikan ke RAGAs
langchain_llm = HuggingFacePipeline(pipeline=gemma_pipeline)

print("Wrapper 'langchain_llm' berhasil dibuat dan siap digunakan untuk evaluasi.")

# ==============================================================================
# TAHAP 1: PERSIAPAN & PENGUMPULAN DATA
# ==============================================================================
# Gabungkan data soal dan data jawaban siswa menjadi satu DataFrame untuk kemudahan
q_df = pd.DataFrame(question_data)
s_df = pd.DataFrame(student_answers_data)
# Lakukan join untuk mendapatkan teks pertanyaan untuk setiap jawaban siswa
eval_prep_df = pd.merge(s_df, q_df, on="question_id")

print(f"Total {len(eval_prep_df)} jawaban akan dievaluasi dalam mode batch.")

# ==============================================================================
# TAHAP 2: EKSEKUSI BATCH
# ==============================================================================
# --- Batch Retrieval ---
print("\nLangkah 1: Melakukan Embedding & Retrieval secara Batch...")
start_retrieval_time = time.time()

# Ambil semua teks pertanyaan untuk di-embed sekaligus
questions_to_embed = eval_prep_df["question_text"].tolist()
# Dapatkan semua embedding dalam satu panggilan
all_embeddings = get_embeddings(questions_to_embed)
# Lakukan pencarian FAISS untuk semua embedding sekaligus
distances, indices = kbindex.search(all_embeddings, k=5)

# Ambil teks dari chunk yang relevan berdasarkan hasil pencarian
all_retrieved_contexts = []
for i in range(len(indices)):
    chunks = [knowledge_base_chunks[idx]["text"] for idx in indices[i] if idx != -1]
    all_retrieved_contexts.append("\n\n".join(chunks))

eval_prep_df["retrieved_context"] = all_retrieved_contexts
end_retrieval_time = time.time()
print(
    f"Embedding & Retrieval selesai dalam {end_retrieval_time - start_retrieval_time:.2f} detik."
)


# --- Batch Generation ---
print("\nLangkah 2: Melakukan Evaluasi oleh Gemma secara Batch...")
# Buat daftar prompt lengkap untuk setiap baris data
prompts = []
for row_index, row in eval_prep_df.iterrows():
    prompt = f"""Anda adalah seorang pengajar bahasa Jepang ahli.
Tugas Anda adalah mengevaluasi jawaban esai siswa. Berikan umpan balik yang konstruktif.

### Pertanyaan:
{row['question_text']}

### Jawaban Siswa:
{row['original_text']}

### Konteks/Referensi:
{row['retrieved_context']}

Silakan berikan evaluasi Anda:"""
    prompts.append(prompt)

# Catat waktu mulai untuk proses generation
start_generation_time = time.time()

# Panggil gemma_pipeline SATU KALI dengan semua prompt
# gemma_pipeline sudah dikonfigurasi dengan batch_size di blok sebelumnya
gemma_evaluations = gemma_pipeline(prompts)

# Catat waktu selesai
end_generation_time = time.time()

# Ekstrak teks yang dihasilkan saja
eval_prep_df["gemma_evaluation"] = [
    result[0]["generated_text"].replace(p, "").strip()
    for p, result in zip(prompts, gemma_evaluations)
]

In [ ]:
# ==============================================================================
# TAHAP 3: PENGUMPULAN HASIL & ANALISIS WAKTU
# ==============================================================================
total_generation_time = end_generation_time - start_generation_time
average_time_per_answer = total_generation_time / len(eval_prep_df)

print(f"\nProses evaluasi batch selesai.")
print(
    f"Total waktu untuk {len(eval_prep_df)} jawaban: {total_generation_time:.2f} detik."
)
print(f"Rata-rata waktu per jawaban: {average_time_per_answer:.2f} detik.")

# Simpan hasil lengkap ke CSV
batch_results_path = os.path.join(KB_PATH, "batch_evaluation_results.csv")
eval_prep_df.to_csv(batch_results_path, index=False)
print(f"\nHasil evaluasi batch lengkap disimpan di: {batch_results_path}")

# Tampilkan cuplikan hasil
print("\nCuplikan Hasil Evaluasi Batch:")
display(eval_prep_df[["student_id", "question_id", "gemma_evaluation"]].head())


# --- Visualisasi Waktu ---
# Membuat DataFrame sederhana untuk plotting waktu rata-rata
timing_summary = {
    "Total Jawaban": len(eval_prep_df),
    "Total Waktu (detik)": total_generation_time,
    "Rata-rata Waktu per Jawaban (detik)": average_time_per_answer,
}

print("\n--- Ringkasan Waktu Evaluasi ---")
print(pd.Series(timing_summary))

# Plot sederhana untuk menunjukkan total vs rata-rata
plt.figure(figsize=(8, 6))
sns.barplot(x=list(timing_summary.keys())[1:], y=list(timing_summary.values())[1:])
plt.title("Ringkasan Kinerja Waktu Evaluasi Batch", fontsize=16)
plt.ylabel("Waktu (detik)", fontsize=12)
plt.show()

In [ ]:
# --- 3.2. Proses Evaluasi untuk Setiap Siswa (Dimodifikasi dengan Pencatatan Waktu) ---

# (1) Impor library tambahan yang diperlukan untuk mencatat waktu dan visualisasi
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("\nMemulai proses evaluasi RAG untuk semua jawaban siswa...")
all_evaluations = {}
# (2) Inisialisasi list untuk menampung data waktu
timing_data = []

# (3) Loop utama untuk evaluasi pada SEMUA jawaban siswa (filter dihapus)
# Kita langsung melakukan iterasi pada `student_answers_data`
for student_ans in tqdm(student_answers_data, desc="Mengevaluasi Semua Jawaban"):
    # Catat waktu mulai untuk satu jawaban
    start_time = time.time()

    student_id = student_ans["student_id"]
    question_id = student_ans["question_id"]

    if student_id not in all_evaluations:
        all_evaluations[student_id] = {}

    try:
        # Mencari teks pertanyaan yang sesuai.
        question_text = next(
            q["question_text"] for q in question_data if q["question_id"] == question_id
        )
    except StopIteration:
        print(
            f"Peringatan: Melewati jawaban untuk {student_id} karena question_id '{question_id}' tidak ditemukan."
        )
        continue

    # --- Proses RAG ---
    # 1. Retrieval
    query_embedding = get_embeddings([question_text])[0]
    retrieved_chunks = retrieve_knowledge(query_embedding)

    # 2. Generation
    gemma_evaluation = generate_gemma_response(
        question_text, student_ans["original_text"], retrieved_chunks
    )

    # --- Gabungkan semua hasil ke dalam dictionary ---
    all_evaluations[student_id][question_id] = {
        "original_student_answer": student_ans["original_text"],
        "retrieved_context": "\n\n".join([c["text"] for c in retrieved_chunks]),
        "simple_o_score": None,
        "gemma_evaluation": gemma_evaluation,
    }

    # (4) Catat waktu selesai dan hitung durasi
    end_time = time.time()
    evaluation_time = end_time - start_time

    # Tambahkan data waktu ke dalam list
    timing_data.append(
        {
            "student_id": student_id,
            "question_id": question_id,
            "evaluation_time_seconds": evaluation_time,
        }
    )

print("\nProses evaluasi RAG selesai.")

# Menyimpan hasil evaluasi utama ke file JSON (seperti kode asli)
output_eval_path = os.path.join(KB_PATH, "all_student_evaluations.json")
with open(output_eval_path, "w", encoding="utf-8") as f:
    json.dump(all_evaluations, f, ensure_ascii=False, indent=4)
print(f"\nSemua hasil evaluasi disimpan di: {output_eval_path}")


# ==============================================================================
# BAGIAN BARU: Analisis Waktu Evaluasi
# ==============================================================================
print("\n--- Analisis Waktu Evaluasi ---")

# (5) Buat DataFrame dari data waktu dan simpan ke CSV
timing_df = pd.DataFrame(timing_data)
timing_csv_path = os.path.join(KB_PATH, "evaluation_timing_report.csv")
timing_df.to_csv(timing_csv_path, index=False)
print(f"Laporan waktu evaluasi telah disimpan di: {timing_csv_path}")

# (6) Tampilkan 5 baris pertama (head) dari DataFrame waktu
print("\nCuplikan Laporan Waktu (5 baris pertama):")
print(timing_df.head())

# Tampilkan statistik deskriptif
print("\nStatistik Waktu Evaluasi (detik):")
print(timing_df["evaluation_time_seconds"].describe())

# (7) Buat dan simpan visualisasi plot
print("\nMembuat visualisasi plot...")
plt.style.use("seaborn-v0_8-whitegrid")
fig, axes = plt.subplots(2, 1, figsize=(12, 14))  # Membuat 2 plot dalam 1 gambar

# Plot 1: Box Plot untuk Distribusi Waktu Keseluruhan
sns.boxplot(x=timing_df["evaluation_time_seconds"], ax=axes[0])
axes[0].set_title("Distribusi Waktu Evaluasi per Jawaban", fontsize=16)
axes[0].set_xlabel("Waktu (detik)", fontsize=12)

# Plot 2: Bar Plot untuk Rata-rata Waktu per Soal
avg_time_per_question = (
    timing_df.groupby("question_id")["evaluation_time_seconds"].mean().sort_index()
)
avg_time_per_question.plot(
    kind="bar",
    ax=axes[1],
    color=sns.color_palette("viridis", len(avg_time_per_question)),
)
axes[1].set_title("Rata-rata Waktu Evaluasi per Soal", fontsize=16)
axes[1].set_xlabel("ID Soal", fontsize=12)
axes[1].set_ylabel("Rata-rata Waktu (detik)", fontsize=12)
axes[1].tick_params(axis="x", rotation=0)

plt.tight_layout(pad=3.0)

# Simpan gambar plot
plot_path = os.path.join(KB_PATH, "evaluation_timing_plot.png")
plt.savefig(plot_path)
print(f"Plot visualisasi waktu telah disimpan di: {plot_path}")

# Tampilkan plot
plt.show()

In [ ]:
# ==============================================================================
# BAGIAN 3.4 (REVISI): PERSIAPAN DATA UNTUK EVALUASI RAGAS
# ==============================================================================
import json
import pandas as pd
from datasets import Dataset

print("Memulai persiapan data untuk evaluasi RAGAs dengan ground truth kustom...")


def load_ground_truth_from_docx(file_path):
    """
    Membaca file .docx yang berisi string JSON, membersihkannya,
    dan mem-parse-nya dengan lebih tangguh.
    """
    try:
        doc = docx.Document(file_path)
        full_text = "\n".join([para.text for para in doc.paragraphs])

        # Membersihkan teks dari spasi berlebih di awal/akhir
        cleaned_text = full_text.strip()

        # ---- PERBAIKAN UTAMA DIMULAI DI SINI ----
        # Jika teks tidak kosong dan belum menjadi array JSON yang valid
        if cleaned_text and not (
            cleaned_text.startswith("[") and cleaned_text.endswith("]")
        ):
            # Bungkus dengan kurung siku untuk menjadikannya array JSON yang valid
            json_array_string = f"[{cleaned_text}]"
        else:
            json_array_string = cleaned_text
        # ---- AKHIR PERBAIKAN ----

        # Coba parse string yang sudah diperbaiki
        return json.loads(json_array_string)

    except FileNotFoundError:
        print(f"ERROR: File ground truth tidak ditemukan di {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"ERROR: Gagal mem-parse JSON dari file {file_path}.")
        print(f"Detail Error: {e}")
        # Tambahan: Tampilkan beberapa karakter pertama dari teks yang bermasalah
        print("Teks yang bermasalah (100 karakter pertama):")
        print(repr(json_array_string[:100]))
        return None
    except Exception as e:
        print(f"Terjadi error saat membaca file ground truth: {e}")
        return None


# Path ke file ground truth Anda
ground_truth_path = os.path.join(KB_PATH, "ground_truth/gt_mahasiswa1.docx")

# Muat data ground truth
ground_truth_list = load_ground_truth_from_docx(ground_truth_path)

# Ubah ground truth menjadi dictionary yang mudah diakses dengan key 'question_id'
if ground_truth_list:
    ground_truth_map = {item["question_id"]: item for item in ground_truth_list}
    print(f"Berhasil memuat {len(ground_truth_map)} data ground truth.")

    # Siapkan data untuk RAGAs
    data_for_ragas = []
    # `all_evaluations` adalah variabel dari hasil eksekusi RAG Anda sebelumnya
    # Kita hanya akan mengevaluasi mahasiswa yang ada di ground truth (M1)
    if "M1" in all_evaluations:
        for question_id, rag_output in all_evaluations["M1"].items():
            if question_id in ground_truth_map:
                gt_data = ground_truth_map[question_id]
                question_text = next(
                    (
                        q["question_text"]
                        for q in question_data
                        if q["question_id"] == question_id
                    ),
                    None,
                )

                if question_text:
                    data_for_ragas.append(
                        {
                            "question": question_text,
                            "answer": rag_output[
                                "gemma_evaluation"
                            ],  # Jawaban dari sistem RAG Anda
                            "contexts": [
                                ctx.strip()
                                for ctx in rag_output["retrieved_context"].split("###")
                                if ctx.strip()
                            ],
                            "ground_truth": gt_data[
                                "gemini_evaluation"
                            ],  # Jawaban ideal dari file GT
                        }
                    )

    if data_for_ragas:
        # Buat dataset
        ragas_dataset = Dataset.from_list(data_for_ragas)
        print("\nDataset untuk evaluasi RAGAs berhasil dibuat:")
        print(ragas_dataset)
        # Konversi ke DataFrame untuk digunakan di blok-blok berikutnya
        evaluation_df = ragas_dataset.to_pandas()
    else:
        print(
            "\nTidak ada data yang cocok antara hasil evaluasi RAG dan ground truth. Evaluasi tidak dapat dilanjutkan."
        )
        evaluation_df = pd.DataFrame()  # Buat DataFrame kosong
else:
    print("\nEvaluasi tidak dapat dilanjutkan karena gagal memuat ground truth.")
    evaluation_df = pd.DataFrame()  # Buat DataFrame kosong

In [ ]:
# ==============================================================================
# BLOK 3.5: EVALUASI GENERATION - FAITHFULNESS
# ==============================================================================
import os

os.environ["RAGAS_DEBUG"] = "true"
from ragas import evaluate
from ragas.metrics import faithfulness
from ragas.llms import LangchainLLMWrapper
from ragas.run_config import RunConfig

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch


# sbert_model_name = 'sonoisa/sentence-bert-base-ja-mean-tokens'

if not evaluation_df.empty:
    print("Mengevaluasi metrik: Faithfulness...")

    gemma_pipeline = pipeline(
        "text-generation",
        model=gemma_model,
        tokenizer=gemma_tokenizer,
        max_new_tokens=128,
        torch_dtype=torch.bfloat16,
        batch_size=16,
        truncation=True,
    )
    langchain_llm = HuggingFacePipeline(pipeline=gemma_pipeline)

    run_config = RunConfig(timeout=60, log_tenacity=True)
    ragas_llm = LangchainLLMWrapper(langchain_llm, run_config)

    # Inisialisasi embeddings
    sbert_model_name = "sentence-transformers/all-mpnet-base-v2"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=sbert_model_name, model_kwargs={"device": "cuda"}
    )

    print("Komponen siap. Mengevaluasi metrik: Faithfulness...")
    print(evaluation_df)

    result_faithfulness = evaluate(
        dataset=Dataset.from_pandas(evaluation_df),
        metrics=[faithfulness],
        llm=ragas_llm,
        embeddings=hf_embeddings,
        raise_exceptions=False,
    )

In [ ]:
# ==============================================================================
# BLOK 3.5.1: UJI COBA DENGAN DATASET MINI UNTUK DEBUGGING
# ==============================================================================
# Blok ini bertujuan untuk menguji alur evaluasi dengan data yang paling
# sederhana untuk melihat apakah error Timeout/KeyError tetap terjadi.

print("--- Memulai Uji Coba dengan Dataset Mini ---")

# (1) Membuat dataset yang sangat pendek dan sederhana
# Strukturnya sama persis, tetapi isinya hanya satu kata.
# Faithfulness harusnya mudah memverifikasi bahwa "Biru" ada di dalam konteks.
mini_data_samples = {
    "question": ["Is blue a color?"],
    "answer": ["Yes it is"],
    "contexts": [["Blue in one of color"]],
    "ground_truth": ["Yes, blues is a color"],  # Tetap sertakan untuk konsistensi
}
mini_dataset = Dataset.from_dict(mini_data_samples)

print("Dataset mini berhasil dibuat:")
print(mini_dataset)

# (2) Menjalankan evaluasi pada dataset mini
# Kita menggunakan kembali `langchain_llm_for_eval` dan `ragas_embeddings_wrapper`
# yang sudah dibuat di blok sebelumnya.
try:
    print("\nMengevaluasi 'faithfulness' pada dataset mini...")

    # Ambil metrik faithfulness yang sudah dikonfigurasi jika ada,
    # atau cukup gunakan metrik mentah jika menjalankan blok ini secara terpisah
    # Untuk keamanan, kita gunakan metrik mentah saja di sini.
    print("Mengevaluasi metrik: Faithfulness...")

    gemma_pipeline = pipeline(
        "text-generation",
        model=gemma_model,
        tokenizer=gemma_tokenizer,
        max_new_tokens=128,
        torch_dtype=torch.bfloat16,
        batch_size=16,
        truncation=True,
    )
    langchain_llm = HuggingFacePipeline(pipeline=gemma_pipeline)

    run_config = RunConfig(timeout=60, log_tenacity=True)
    ragas_llm = LangchainLLMWrapper(langchain_llm, run_config)

    # Inisialisasi embeddings
    sbert_model_name = "sentence-transformers/all-mpnet-base-v2"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=sbert_model_name, model_kwargs={"device": "cuda"}
    )

    print("Komponen siap. Mengevaluasi metrik: Faithfulness...")

    result_faithfulness = evaluate(
        dataset=mini_dataset,
        metrics=[faithfulness],
        llm=ragas_llm,
        embeddings=hf_embeddings,
        raise_exceptions=False,
    )

except Exception as e:
    print(f"\nUji coba GAGAL dengan error: {e}")

print("\n--- Uji Coba Selesai ---")

In [ ]:
faithfulness_df = result_faithfulness.to_pandas()
print("Evaluasi Faithfulness Selesai.")
print(faithfulness_df[["question", "answer", "faithfulness_score"]].head())

if "faithfulness_score" in faithfulness_df.columns:
    evaluation_df = pd.merge(
        evaluation_df,
        faithfulness_df[["question", "faithfulness_score"]],
        on="question",
        how="left",  # Gunakan left merge untuk menjaga semua baris asli
    )
else:
    print(
        "Warning: 'faithfulness_score' column not found in faithfulness_df after evaluation."
    )

In [ ]:
# ==============================================================================
# BLOK 3.6: UJI COBA 'ANSWER_RELEVANCY' (DENGAN MODEL GEMMA LOKAL)
# ==============================================================================
import os
import torch
from datasets import Dataset
from ragas import evaluate

# Impor metrik yang akan diuji
from ragas.metrics import answer_relevancy

# Impor semua wrapper dan komponen yang diperlukan
from ragas.run_config import RunConfig
from ragas.embeddings import LangchainEmbeddingsWrapper
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings

if not evaluation_df.empty:
    print("--- Memulai Uji Coba Metrik: Answer Relevancy (dengan Gemma Lokal) ---")

    try:
        # (1) Konfigurasi Eksekusi
        run_config = RunConfig(
            timeout=600, log_tenacity=True
        )  # Gunakan timeout panjang untuk keamanan

        # (2) Penyiapan LLM (sesuai format yang Anda minta)
        gemma_pipeline = pipeline(
            "text-generation",
            model=gemma_model,
            tokenizer=gemma_tokenizer,
            max_new_tokens=512,
            torch_dtype=torch.bfloat16,
            batch_size=16,
        )

        print("\nMengevaluasi 'answer_relevancy'...")
        result_relevancy = evaluate(
            dataset=Dataset.from_pandas(evaluation_df),
            metrics=[answer_relevancy],
            llm=ragas_llm,
            embeddings=hf_embeddings,
            raise_exceptions=False,
        )

        relevancy_df = result_relevancy.to_pandas()
        print("\nEvaluasi 'answer_relevancy' SELESAI.")
        print("Hasil:")
        display(relevancy_df)

        # (Opsional) Gabungkan skor ke DataFrame utama
        score_col = "answer_relevancy_score"
        if score_col in relevancy_df.columns:
            evaluation_df = pd.merge(
                evaluation_df,
                relevancy_df[["question", score_col]],
                on="question",
                how="left",
            )
            print(f"\nSkor '{score_col}' berhasil digabungkan.")

    except Exception as e:
        print(f"\nUji coba 'answer_relevancy' GAGAL dengan error: {e}")

else:
    print("DataFrame evaluasi kosong, melewati blok 3.6.")

In [ ]:
# ==============================================================================
# BLOK 3.8: EVALUASI RETRIEVAL - CONTEXT RECALL
# ==============================================================================
from ragas.metrics import context_recall

if not evaluation_df.empty:
    print("\nMengevaluasi metrik: Context Recall...")
    result_recall = evaluate(
        dataset=Dataset.from_pandas(evaluation_df),
        metrics=[context_recall],
        raise_exceptions=False,
    )
    recall_df = result_recall.to_pandas()
    print("Evaluasi Context Recall Selesai.")
    print(recall_df[["question", "ground_truth", "context_recall_score"]].head())

    # Gabungkan hasil ke DataFrame utama
    evaluation_df = pd.merge(
        evaluation_df, recall_df[["question", "context_recall_score"]], on="question"
    )
else:
    print("DataFrame evaluasi kosong, melewati blok 3.8.")

In [ ]:
# ==============================================================================
# BLOK 3.7: EVALUASI RETRIEVAL - CONTEXT PRECISION
# ==============================================================================
from ragas.metrics import context_precision

if not evaluation_df.empty:
    print("\nMengevaluasi metrik: Context Precision...")
    result_precision = evaluate(
        dataset=Dataset.from_pandas(evaluation_df),
        metrics=[context_precision],
        raise_exceptions=False,
    )
    precision_df = result_precision.to_pandas()
    print("Evaluasi Context Precision Selesai.")
    print(precision_df[["question", "contexts", "context_precision_score"]].head())

    # Gabungkan hasil ke DataFrame utama
    evaluation_df = pd.merge(
        evaluation_df,
        precision_df[["question", "context_precision_score"]],
        on="question",
    )
else:
    print("DataFrame evaluasi kosong, melewati blok 3.7.")

In [ ]:
# ==============================================================================
# BLOK 3.9: AGREGASI, VISUALISASI, DAN PENYIMPANAN HASIL
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

if not evaluation_df.empty and "faithfulness_score" in evaluation_df.columns:
    print("\n--- HASIL AKHIR EVALUASI RAG ---")

    # Pilih kolom skor untuk ditampilkan
    score_columns = [
        "faithfulness_score",
        "answer_relevancy_score",
        "context_precision_score",
        "context_recall_score",
    ]
    final_results_df = evaluation_df[["question"] + score_columns]

    print("\nTabel Hasil Akhir:")
    print(final_results_df)

    # Menghitung skor rata-rata
    average_scores = final_results_df[score_columns].mean().reset_index()
    average_scores.columns = ["Metric", "Average Score"]

    print("\nSkor Rata-Rata per Metrik:")
    print(average_scores)

    # --- Visualisasi Data ---
    print("\nMembuat visualisasi hasil evaluasi...")
    plt.style.use("seaborn-v0_8-whitegrid")
    fig, ax = plt.subplots(figsize=(10, 6))

    sns.barplot(
        x="Average Score", y="Metric", data=average_scores, ax=ax, palette="viridis"
    )

    ax.set_title("Rata-Rata Skor Metrik Evaluasi RAG", fontsize=16)
    ax.set_xlabel("Rata-Rata Skor", fontsize=12)
    ax.set_ylabel("Metrik", fontsize=12)
    ax.set_xlim(0, 1)  # Skor RAGAs berada dalam rentang 0 hingga 1

    # Menambahkan label pada bar
    for i in ax.containers:
        ax.bar_label(i, fmt="%.3f", padding=3)

    plt.tight_layout()

    # Simpan gambar
    figure_path = os.path.join(KB_PATH, "ragas_evaluation_summary.png")
    plt.savefig(figure_path)
    print(f"Gambar grafik telah disimpan di: {figure_path}")
    plt.show()

    # --- Penyimpanan Hasil Akhir ---
    # Simpan DataFrame lengkap ke file CSV
    output_csv_path = os.path.join(KB_PATH, "ragas_full_evaluation_results.csv")
    evaluation_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(
        f"\nLaporan evaluasi lengkap telah disimpan dalam format CSV di: {output_csv_path}"
    )

else:
    print("\nTidak ada hasil untuk divisualisasikan atau disimpan.")